# Recommendation Formulation

In [1]:
import pandas as pd
import os
import json
import logging

from utils.General.quicker import Quicker, QuickerData, QuickerStage
from utils.Evidence_Assessment.paper import Paper
from utils.Evidence_Assessment.outcome import Outcome
from utils.Evidence_Assessment.evidence import Evidence
from utils.logging import setup_logging

setup_logging()

# Hyperparameters
YOUR_CONFIG_PATH = 'YOUR_CONFIG_PATH' # e.g. 'config\config.json'
YOUR_QUESTION_DECOMPOSTITION_PATH =  'YOUR_QUESTION_DECOMPOSTITION_PATH' # e.g. 'data\2021ACR RA\Question_Decomposition'
YOUR_LITERATURE_SEARCH_PATH = 'YOUR_LITERATURE_SEARCH_PATH' # e.g. 'data\2021ACR RA\Literature_Search'
YOUR_STUDY_SELECTION_PATH = 'YOUR_STUDY_SELECTION_PATH' # e.g. 'data\2021ACR RA\Study_Selection'
YOUR_EVIDENCE_ASSESSMENT_PATH = 'YOUR_EVIDENCE_ASSESSMENT_PATH' # e.g. 'data\2021ACR RA\Evidence_Assessment'
YOUR_PAPER_LIBRARY_PATH = 'YOUR_PAPER_LIBRARY_PATH' # e.g. 'data\2021ACR RA\Paper_Library'
YOUR_RECOMMENDATION_FORMATION_PATH = 'YOUR_RECOMMENDATION_FORMATION_PATH' # e.g. 'data\2021ACR RA\Recommendation_Formation'


disease = 'disease_or_topic_of_your_question' # Disease name or clinical topic of your clinical question. e.g. 'Rheumatoid Arthritis (RA)'
pico_idx = "xxxxxxxxx" # PICO index of the clinical question, saved in your PICO_Information.json file
overall_certainty = "LOW" # Overall certainty of the evidence, e.g. "VERY LOW", "LOW", "MODERATE", "HIGH". You can determine this based on the evidence assessment results.
supplementary_information = '' # Any supplementary information can guide LLM to formulate satisfactory recommendations.

In [ ]:
import shutil

# move the files from the source directory to the target directory
def transfer_outcome_and_paperinfo(source_dir_path, target_dir_path, only_index: str = None):
    '''
    Transfer the 'outcomeinfo' and 'paperinfo' folders and their subfolders/files from source_dir_path to target_dir_path.
    If only_index is provided, only transfer subfolders/files containing '_PICO{only_index}' in their names.

    :param source_dir_path: Source directory path
    :param target_dir_path: Target directory path
    :param only_index: Only transfer subfolders/files containing '_PICO{only_index}' in their names
    '''

    # Transfer 'outcomeinfo' folder
    source_outcomeinfo_path = os.path.join(source_dir_path, 'outcomeinfo')
    target_outcomeinfo_path = os.path.join(target_dir_path, 'outcomeinfo')
    if os.path.exists(source_outcomeinfo_path):
        if only_index:
            for item in os.listdir(source_outcomeinfo_path):
                if f'_PICO{only_index}' in item:
                    source_item_path = os.path.join(source_outcomeinfo_path, item)
                    target_item_path = os.path.join(target_outcomeinfo_path, item)
                    if os.path.isdir(source_item_path):
                        shutil.copytree(source_item_path, target_item_path, dirs_exist_ok=True)
                    else:
                        os.makedirs(target_outcomeinfo_path, exist_ok=True)
                        shutil.copy2(source_item_path, target_item_path)
        else:
            shutil.copytree(source_outcomeinfo_path, target_outcomeinfo_path, dirs_exist_ok=True)

    # Transfer 'paperinfo' folder
    source_paperinfo_path = os.path.join(source_dir_path, 'paperinfo')
    target_paperinfo_path = os.path.join(target_dir_path, 'paperinfo')
    if os.path.exists(source_paperinfo_path):
        if only_index:
                    for item in os.listdir(source_paperinfo_path):
                        if f'_PICO{only_index}' in item:
                            source_item_path = os.path.join(source_paperinfo_path, item)
                            target_item_path = os.path.join(target_paperinfo_path, item)
                            if os.path.isdir(source_item_path):
                                shutil.copytree(source_item_path, target_item_path, dirs_exist_ok=True)
                            else:
                                os.makedirs(target_paperinfo_path, exist_ok=True)
                                shutil.copy2(source_item_path, target_item_path)
        else:
            shutil.copytree(source_paperinfo_path, target_paperinfo_path, dirs_exist_ok=True)


In [ ]:
# The confirmed documents need to be transferred to the recommendation formulation path for subsequent operations. Here is an example code to transfer the files.
# transfer_outcome_and_paperinfo(source_dir_path=YOUR_EVIDENCE_ASSESSMENT_PATH,target_dir_path=YOUR_RECOMMENDATION_FORMATION_PATH,only_index=pico_index)

In [3]:
evidence_assessment_results = {}
for file in os.listdir(YOUR_RECOMMENDATION_FORMATION_PATH):
    if file.startswith('outcomeinfo_PICO' + pico_idx ) and file.endswith(
        '.json'
    ):
        file_path = os.path.join(YOUR_RECOMMENDATION_FORMATION_PATH, file)
        with open(file_path, 'r') as f:
            result = json.load(f)

        evidence_assessment_results[result[0]['comparator']] = {
            "outcome_list": result,
            "overall_certainty": overall_certainty,
        }

In [5]:
quicker_data = QuickerData(disease=disease,pico_idx=pico_idx)
quicker = Quicker(
    config_path=YOUR_CONFIG_PATH,
    question_deconstruction_database_path=YOUR_QUESTION_DECOMPOSTITION_PATH,
    literature_search_database_path=YOUR_LITERATURE_SEARCH_PATH,
    study_selection_database_path=YOUR_STUDY_SELECTION_PATH,
    evidence_assessment_database_path=YOUR_EVIDENCE_ASSESSMENT_PATH,
    quicker_data=quicker_data,
    paper_library_base=YOUR_PAPER_LIBRARY_PATH,
)


question_deconstruction_datapath = os.path.join(
    quicker.question_deconstruction_database_path, 'PICO_Information.json'
)
question_deconstruction_data = pd.read_json(
    question_deconstruction_datapath, dtype={'Index': str}
)
question_deconstruction_data = question_deconstruction_data[
    question_deconstruction_data['Index'] == pico_idx
]
original_qd_dict = question_deconstruction_data.to_dict(orient='records')

data_dict = dict(
    clinical_question=original_qd_dict[0]['Question'],
    evidence_assessment_results=evidence_assessment_results,
)

quicker._add_data_to_quickerdata_for_test(
    stage=QuickerStage.EVIDENCE_ASSESSMENT,
    default_value=data_dict,
)


quicker.update_input_data(dict(supplementary_information=supplementary_information))

print(quicker)
# print(quicker.quicker_data)
print(quicker.quicker_data.check_stage_state())
print(quicker.quicker_data.not_none_data)


Quicker: RECOMMENDATION_FORMATION
StageState.NOT_STARTED
{'disease': 'Rheumatoid Arthritis (RA)', 'clinical_question': 'Should Methotrexate or leflunomide be the first choice for initial treatment of rheumatoid arthritis?', 'pico_idx': 'c3072c89', 'population': '', 'intervention': '', 'comparison': [], 'study': [], 'outcome': {}, 'search_config': {}, 'annotation': {}, 'search_strategy': '', 'search_results': {}, 'inclusion_criteria': '', 'exclusion_criteria': '', 'record_included_studies': [], 'full_text_included_studies': [], 'total_outcome_list': [], 'valid_comparison_list': [], 'outcome_list': [], 'paper_list': [], 'evidence_assessment_results': {'Leflunomide (LEF)': {'outcome_list': [{'outcome_uid': '43b26894', 'clinical_question': 'Should Methotrexate or leflunomide be the first choice for initial treatment of rheumatoid arthritis?', 'population': 'Treatment-naïve patients with rheumatoid arthritis', 'intervention': 'Methotrexate (MTX)', 'comparator': 'Leflunomide (LEF)', 'outcome

In [6]:
final_result = quicker.form_recommendation(
    evidence_assessment_results=quicker.quicker_data.evidence_assessment_results
    )


quicker.quicker_data.update_data(dict(final_result=final_result))

In [7]:
quicker.quicker_data.to_json(YOUR_RECOMMENDATION_FORMATION_PATH)